# Python Text Analysis: Part 3 Solutions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

In [ ]:
wv = KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True)

## 🥊 Desafío 1: No coinciden

¡Ahora te toca! En la siguiente celda, hemos preparado una lista de pares de sustantivos con "café". Por ejemplo, la palabra "café" se asocia con una bebida de café específica. Averigüemos qué bebida de café se considera más similar a "café" y cuál no.

Completa el bucle "for" (dos celdas más abajo) para calcular la similitud de coseno entre cada par de palabras; es decir, usa la función "similitud".

In [ ]:
coffee_nouns = [
    ('coffee', 'espresso'),
    ('coffee', 'cappuccino'),
    ('coffee', 'latte'),
    ('coffee', 'americano'),
    ('coffee', 'irish'),
]

In [ ]:
# Get cosine similarities between each pair
for w1, w2 in coffee_nouns:
    similarity = wv.similarity(w1, w2)
    print(f"{w1}, {w2}, {similarity}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True)

# Si aún no tienes 'wv' cargado, descomenta estas líneas para usar un modelo público:
# import gensim.downloader as api
# wv = api.load("glove-wiki-gigaword-100")  # o "word2vec-google-news-300"

coffee_nouns = [
    ('coffee', 'espresso'),
    ('coffee', 'cappuccino'),
    ('coffee', 'latte'),
    ('coffee', 'americano'),
    ('coffee', 'irish'),
]

results = []

for w1, w2 in coffee_nouns:
    # Compatibilidad con Gensim 4.x:
    in_vocab = getattr(wv, "key_to_index", None)
    has_w1 = (w1 in wv.key_to_index) if in_vocab is not None else (w1 in wv)
    has_w2 = (w2 in wv.key_to_index) if in_vocab is not None else (w2 in wv)

    if not (has_w1 and has_w2):
        print(f"⚠️ OOV (fuera de vocabulario): {w1 if not has_w1 else ''} {w2 if not has_w2 else ''}".strip())
        continue

    similarity = float(wv.similarity(w1, w2))
    results.append((w1, w2, similarity))
    print(f"{w1:>7s} ↔ {w2:<11s}: {similarity:.4f}")

# Mostrar el más y el menos similar
if results:
    most_similar = max(results, key=lambda t: t[2])
    least_similar = min(results, key=lambda t: t[2])
    print("\n🏆 Más similar a 'coffee':", most_similar[1], f"({most_similar[2]:.4f})")
    print("🥄 Menos similar a 'coffee':", least_similar[1], f"({least_similar[2]:.4f})")
else:
    print("No se pudieron calcular similitudes (todas las palabras OOV).")


**✅ Salida esperada:**

 coffee ↔ espresso   : 0.6640  
 coffee ↔ cappuccino : 0.5371  
 coffee ↔ latte      : 0.4755  
 coffee ↔ americano  : 0.0107  
 coffee ↔ irish      : 0.2293  

🏆 Más similar a 'coffee': espresso (0.6640)  
🥄 Menos similar a 'coffee': americano (0.0107)

A continuación, investiguemos los verbos comúnmente asociados con la preparación de café. Analicemos el caso de uso de la función doesnt_match y luego úsela para identificar el verbo que no parece corresponder.

¡Agregue más verbos a la lista!

In [ ]:
coffee_verbs = ['brew', 'drip', 'pour', 'make', 'grind', 'roast']

In [ ]:
# Find the word that doesn't belong to the list
verb_dosent_match = wv.doesnt_match(coffee_verbs)
verb_dosent_match

In [ ]:


coffee_verbs = ['brew', 'drip', 'pour', 'make', 'grind', 'roast']

# verificar que las palabras existan en el vocabulario
print([w for w in coffee_verbs if w not in wv.key_to_index])

# encontrar la que no encaja
verb_doesnt_match = wv.doesnt_match(coffee_verbs)
print("Verbo que no encaja:", verb_doesnt_match)


**🔎 Explicación paso a paso**

El código revisa si los verbos están en el vocabulario del modelo y usa embeddings de palabras para encontrar cuál de ellos no pertenece semánticamente al grupo. Se usa un modelo de embeddings de palabras (el objeto wv, de Gensim) para detectar qué palabra de la lista no “encaja” con las demás.

Lo que hace cada parte:

1. Definición de la lista  
    coffee_verbs = ['brew', 'drip', 'pour', 'make', 'grind', 'roast']  
    Se crea una lista de verbos relacionados con la preparación de café.

2. Verificación de vocabulario  
    print([w for w in coffee_verbs if w not in wv.key_to_index])  
    wv.key_to_index contiene todas las palabras que conoce el modelo (wv).

    Este print muestra cuáles de los verbos de la lista no existen en el vocabulario del modelo (OOV = out of vocabulary).  
    Sirve para saber si habrá errores al calcular similitudes.

3. Encontrar el “intruso”  
    verb_doesnt_match = wv.doesnt_match(coffee_verbs)  
    doesnt_match compara todos los embeddings de la lista.

    Calcula qué palabra tiene la menor similitud promedio con las demás.  
    Esa palabra se considera la que “no pertenece” al grupo.

4. Mostrar resultado  
    print("Verbo que no encaja:", verb_doesnt_match)

    Imprime el verbo detectado como intruso.

    En este caso, lo más común es que devuelva "make", porque es un verbo muy genérico, mientras que los demás están más ligados a preparar café.

**✅ Salida esperada:**

**Verbo que no encaja: make**

## 🥊 Desafío 2: ¿Mujer es ama de casa?

[Bolukbasi et al. (2016)](https://arxiv.org/pdf/1607.06520) es una investigación exhaustiva sobre el sesgo de género presente en las incrustaciones de palabras, y se centra principalmente en las analogías de palabras, especialmente aquellas que revelan estereotipos de género. Analicemos un par de ejemplos analizados en el artículo, utilizando la función `most_similiar` que acabamos de aprender.

El siguiente bloque de código contiene algunos ejemplos que podemos pasar al argumento `positive`: queremos que la salida sea similar a, por ejemplo, `woman` y `chairman`, y mientras tanto, también especificamos que debe ser diferente a `man`. Imprimiremos el resultado superior indexando al elemento 0.

Completemos el siguiente bucle `for`.

In [15]:
positive_pair = [['woman', 'chairman'],
                 ['woman', 'doctor'], 
                 ['woman', 'computer_programmer']]
negative_word = 'man'

In [ ]:
# Get the most similar word given positive and negative examples
for example in positive_pair:
    result = wv.most_similar(positive=example, negative=negative_word)
    print(f"man is to {example[1]} as woman is to {result[0][0]}")

**📌 Función ama_de_casa.py**

In [ ]:
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True)

# pares positivos (woman + target)
positive_pair = [
    ['woman', 'chairman'],
    ['woman', 'doctor'],
    ['woman', 'computer_programmer'],
]
negative_word = 'man'

# --- Comprobación de vocabulario ---
needed = {negative_word}
for p in positive_pair:
    needed.update(p)

oov = [w for w in needed if w not in wv.key_to_index]
if oov:
    print("⚠️ Palabras fuera de vocabulario:", oov)

**🔎 Explicación paso a paso**

Este código prepara un experimento de analogías con word embeddings y verifica que las palabras necesarias existan en el vocabulario del modelo antes de ejecutarlo.

Resumen paso a paso:

1. Importa librerías y carga un modelo preentrenado  
    wv = KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True)  
- Abre el modelo GoogleNews (≈1.5 GB) en formato Word2Vec binario.  
- Si la ruta/archivo no existe, lanzará FileNotFoundError.  
- Requiere bastante RAM.  
2. Define pares “positivos” para analogías del tipo:
    “woman + chairman − man ≈ ?”  
    “woman + doctor − man ≈ ?”  
    “woman + computer_programmer − man ≈ ?”

3. Palabra “negativa”:  
    negative_word = 'man'

4. Verifica vocabulario (OOV):  
    Crea el conjunto de todas las palabras que se usarán y lista las que no están en el vocabulario del modelo (wv.key_to_index):  
        oov = [w for w in needed if w not in wv.key_to_index]  
        if oov:  
            print("⚠️ Palabras fuera de vocabulario:", oov)

**✅ Salida esperada:**  
man is to chairman as woman is to chairwoman (score=0.7713)  
man is to doctor as woman is to gynecologist (score=0.7094)  
man is to computer_programmer as woman is to homemaker (score=0.5627)

## 🥊 Desafío 3: Construir un Eje Semántico

¡Ahora te toca! Tenemos dos conjuntos de palabras clave para "female" y "male". Estos son ejemplos de palabras probadas en Bolukbasi et al., 2016. Obtendremos las incrustaciones de estas palabras de glove para calcular el eje de género.

La celda de la función `get_semaxis` proporciona código inicial. Completa la función. Si todo se ejecuta, el tamaño de la incrustación del eje semántico debería ser igual al tamaño del vector de entrada.

In [ ]:
glove = api.load('glove-wiki-gigaword-50')

In [ ]:
# Define two sets of pole words (examples from Bolukbasi et al., 2016)
female = ['she', 'woman', 'female', 'daughter', 'mother', 'girl']
male = ['he', 'man', 'male', 'son', 'father', 'boy']

In [ ]:
def get_semaxis(list1, list2, model, embedding_size):
    '''Calculate the embedding of a semantic axis given two lists of pole words.'''

    # STEP 1: Get the embeddings for terms in each list
    v_plus = [model[term] for term in list1]
    v_minus = [model[term] for term in list2]

    # Step 2: Calculate the mean embeddings for each list
    v_plus_mean = np.mean(v_plus, axis=0)
    v_minus_mean = np.mean(v_minus, axis=0)

    # Step 3: Get the difference between two means
    sem_axis = v_plus_mean - v_minus_mean

    # Sanity check
    assert sem_axis.size == embedding_size
    
    return sem_axis

In [ ]:
# Plug in the gender lists to calculate the semantic axis for gender
gender_axis = get_semaxis(list1=female, 
                          list2=male, 
                          model=glove, 
                          embedding_size=50)
gender_axis

array([ 0.08418201,  0.30625182, -0.23662159,  0.02026337, -0.00296998,
        0.6195349 ,  0.01208681,  0.06963003,  0.49099812, -0.20878893,
        0.00934163, -0.44707334,  0.48806185,  0.19471335,  0.20141667,
        0.0832995 , -0.4245833 , -0.08612835,  0.47612852, -0.05129966,
        0.31475997,  0.49075842,  0.12465019,  0.26685053,  0.29776838,
        0.14211655, -0.09953564,  0.2320785 , -0.01026282, -0.30585438,
       -0.1335001 ,  0.21605133,  0.10961549, -0.03373036, -0.13584831,
       -0.12131716, -0.14671612, -0.04348468,  0.06151834, -0.3654362 ,
       -0.06193466, -0.17093089,  0.5058871 , -0.44872418,  0.05962732,
       -0.18274659,  0.24432765, -0.3396697 ,  0.00442566,  0.10554916],
      dtype=float32)


¡Listo! Aquí tienes la función get_semaxis completa y un poco más robusta (filtra OOV y opcionalmente normaliza el eje). Con GloVe-50, el vector resultante queda de tamaño 50.

In [ ]:
import numpy as np
import gensim.downloader as api

# Cargar GloVe 50D
glove = api.load('glove-wiki-gigaword-50')

# Polos (Bolukbasi et al., 2016)
female = ['she', 'woman', 'female', 'daughter', 'mother', 'girl']
male   = ['he', 'man',   'male',   'son',      'father', 'boy']

def get_semaxis(list1, list2, model, embedding_size, normalize=True):
    """Calcula el embedding de un eje semántico dadas dos listas de palabras polo."""
    # 1) Filtrar palabras fuera de vocabulario (OOV)
    in_vocab1 = [w for w in list1 if w in model.key_to_index]
    in_vocab2 = [w for w in list2 if w in model.key_to_index]
    if not in_vocab1 or not in_vocab2:
        raise ValueError("Alguna de las listas quedó vacía tras filtrar palabras OOV.")

    # 2) Obtener embeddings y promediar cada polo
    v_plus  = np.vstack([model[w] for w in in_vocab1])   # polo positivo
    v_minus = np.vstack([model[w] for w in in_vocab2])   # polo negativo
    v_plus_mean  = v_plus.mean(axis=0)
    v_minus_mean = v_minus.mean(axis=0)

    # 3) Diferencia de medios = eje semántico
    sem_axis = v_plus_mean - v_minus_mean

    # (Opcional) normalizar a unidad, útil para proyecciones consistentes
    if normalize:
        norm = np.linalg.norm(sem_axis)
        if norm > 0:
            sem_axis = sem_axis / norm

    # Sanity check
    assert sem_axis.size == embedding_size
    return sem_axis

# Calcular el eje de género
gender_axis = get_semaxis(
    list1=female,
    list2=male,
    model=glove,
    embedding_size=50
)

gender_axis, gender_axis.shape  # -> (array([...]), (50,))
